# Prerequisites (Download data, code and checkpoints. Preprocess)

In [0]:
!git clone https://github.com/google-research/noisystudent
!cd noisystudent && mkdir ckpt
!cd noisystudent && wget https://storage.googleapis.com/noisystudent/ckpts/svhn/teacher_ckpt.tar.gz -O ckpt/teacher_ckpt.tar.gz
!cd noisystudent/ckpt && tar xzvf teacher_ckpt.tar.gz

!cd noisystudent && python proc_svhn.py \
  --task_name=svhn \
  --raw_data_dir=data/svhn/raw \
  --output_dir=data/svhn/proc


# Run training


In [0]:
# Remember to set "Runtime -> Change runtime type" in the menu to use GPU. 
!cd noisystudent && python main.py \
    --model_name=efficientnet-b0 \
    --use_tpu=False \
    --use_bfloat16=False \
    --task_name=svhn \
    --mode=train \
    --train_batch_size=128 \
    --iterations_per_loop=1000 \
    --save_checkpoints_steps=1000 \
    --unlabel_ratio=5 \
    --teacher_softmax_temp=1 \
    --augment_name=v1 \
    --randaug_mag=5 \
    --final_base_lr=0.0002 \
    --weight_decay=1e-4 \
    --label_data_dir=./data/svhn/proc \
    --teacher_model_name=efficientnet-b0 \
    --teacher_model_path=ckpt/teacher_ckpt/model.ckpt \
    --model_dir=./ckpt/exp_1 \
    --unlabel_data_dir=./data/svhn/proc/unlabeled

# Run evaluation

In [0]:
!cd noisystudent && python main.py \
    --model_name=efficientnet-b0 \
    --use_tpu=False \
    --use_bfloat16=False \
    --task_name=svhn \
    --mode=eval \
    --label_data_dir=./data/svhn/proc \
    --model_dir=./ckpt/exp_1